<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#env-set-up" data-toc-modified-id="env-set-up-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>env set-up</a></span></li></ul></div>

# env set-up

In [1]:
source("../code/func.R") ## import lib, model, func
rAw = read.csv("../result/maxYield_all.csv", header=T) ## analytical scan
dIs = read.csv("../result/discrepancy.csv", header = T) ## discrepancies scan (analytical solve vs integration)